In [1]:

from sae_lens import HookedSAETransformer, SAE, SAEConfig
from gemma_utils import get_gemma_2_config, gemma_2_sae_loader

In [67]:

import numpy as np
import torch
import tqdm
import einops
import re
from jaxtyping import Int, Float
from typing import List, Optional, Any
from torch import Tensor
import json
import os
from torch.utils.data import Dataset, DataLoader
import random
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import random
from transformer_lens.utils import get_act_name
from IPython.display import display, HTML

In [3]:

np.random.seed(0)
random.seed(0)
torch.random.manual_seed(0)

In [4]:

model = HookedSAETransformer.from_pretrained("google/gemma-2-2b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


In [91]:
text = """<body><h1>List of My Brother's Favourite Cities</h1><ul><li>"""
tokens = model.to_tokens(text)
out = model.generate(
    tokens,
    max_new_tokens = 100,
    temperature = 0.7,
    top_p = 0.9,
    stop_at_eos=True,
    )

visualize_html(model.to_string(out[-1]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [70]:
def visualize_html(text):
    text = text.replace("<eos>","")
    text = text.replace("<bos>","")
    display(HTML(text))
    

In [94]:
string = "<bos><body><h1>List of My Brother's Favourite Cities</h1><ul><li>Bangkok, Thailand</li><li>London, England</li><li>Paris, France</li><li>Melbourne, Australia</li><li>Toronto, Canada</li><li>New York, USA</li></ul></body>\n<eos>"
tokens = model.to_tokens(string, prepend_bos=False)

At this point and with the changes that we have made to the setup to reliably produce list that don't spiral out of control, we ask ourselves how can the model decide to add another elemnt or not.


Taking into account the moel's tokenization:

- `'<ul>' '<', 'ul', '>'`
- `'><li>' '><', 'li', '>'`
- `'</li>' '</', 'li', '>'`
- `'><ul>' '></', 'ul', '>'`

The model will continue the list while remaining html consistent if it uses the token `><` instead of the token `></`

In [148]:
tok_id = model.to_tokens("></", prepend_bos=False)[0]

tok_id2 = model.to_tokens("><", prepend_bos=False)[0]
tok_pos = torch.where(out[0] == tok_id)[0][0].tolist()
all_tok2_pos = torch.where(out[0] == tok_id2)[0][1:].tolist()

In [121]:
with torch.no_grad():
    logits = model(tokens)

In [153]:
for elem in all_tok2_pos + [tok_pos]:
    l = logits[0,elem]
    print(l[tok_id]-l[tok_id2])

tensor([-2.7107], device='cuda:0')
tensor([-2.0631], device='cuda:0')
tensor([-1.3211], device='cuda:0')
tensor([-1.1420], device='cuda:0')
tensor([-0.5403], device='cuda:0')
tensor([1.1881], device='cuda:0')


In [161]:
(l.log_softmax(dim = -1)).topk(5)

torch.return_types.topk(
values=tensor([-3.6416e-03, -6.0267e+00, -8.7354e+00, -8.9372e+00, -9.0377e+00],
       device='cuda:0'),
indices=tensor([601, 515, 561, 727, 913], device='cuda:0'))